In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
image_dir = "./dataset/images"

image_names = os.listdir(image_dir)
images = [os.path.join(image_dir, name) for name in image_names]

#for i in images[:100]:
#    print(i)

In [3]:
image_name_queue = tf.train.string_input_producer(images, shuffle=False)
label_name_queue = tf.train.string_input_producer(['./dataset/label/Label.csv'], shuffle=False)

image_reader = tf.WholeFileReader()
key, raw_image = image_reader.read(image_name_queue) #key는 filename, raw_image는 실제 image data(2진 데이터임)

csv_reader = tf.TextLineReader()
key, raw_txt = csv_reader.read(label_name_queue)

png_image = tf.image.decode_png(raw_image) #컬러 이미지는 r, g, b로 3차원 Matix 형성
csv_label = tf.decode_csv(raw_txt, record_defaults=[[0]])

png_image = tf.reduce_mean(png_image, axis=2) #실습에서는 흑백 이미지 이기 때문에 차원을 2차로 축소
png_image = tf.reshape(png_image, [61, 49, 1]) #image size는 높이 * 넓이, 1은 3차원을 만들기 위해 강제로 넣은 값

#x, y_ = tf.train.batch([png_image, csv_label], 32) #32는 mini batch size
x, y_ = tf.train.shuffle_batch([png_image, csv_label], batch_size=32, capacity=50000, min_after_dequeue=10000)
#순차적으로 들어간 값(label)을 가져와서 Training 하면 일부 데이터로 훈련하기 때문에 suffle을 해야 함

#print(x.shape)

x = tf.cast(x, tf.float32) #x의 값이 integer여서 연산을 위해 float32로 변환

#print(y_.shape)
y_ = tf.reshape(y_, [-1])
#print(y_.shape)

y_ = tf.one_hot(y_, depth=3, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32) #이해가 안됨
#y_ = tf.reduce_mean(y_, axis=2)

conv1 = tf.layers.conv2d(x, filters=13, kernel_size=[3, 3], activation=tf.nn.relu, padding="SAME") #일반적으로 kernel_size는 3*3을 선호한다.
pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=[2, 2]) #pool_size를 [2, 2]로 써도 무방함

#print(pool1.shape)
tf.summary.image('feature_map', tf.reduce_mean(pool1, axis=3, keep_dims=True))

conv2 = tf.layers.conv2d(pool1, filters=20, kernel_size=[3, 3], activation=tf.nn.relu, padding="SAME") #kernel_size는 3을 써도 무방
pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=[2, 2]) 

conv3 = tf.layers.conv2d(pool2, filters=30, kernel_size=[3, 3], activation=tf.nn.relu, padding="SAME") #일반적으로 kernel_size는 3*3을 선호한다.
pool3 = tf.layers.max_pooling2d(conv3, pool_size=2, strides=[2, 2]) #pool_size를 [2, 2]로 써도 무방함

fc_input_size = int(pool3.shape[1]) * int(pool3.shape[2]) * int(pool3.shape[3]) #이해가 안됨
flat = tf.reshape(pool3, shape=[-1, fc_input_size]) #-1을 넣은 이유는 3차원 큐브가 batch로 인해 32개가 존재하는데
                                                    #이것까지 input으로 넣으면 안되기 때문임

drop_prob = tf.placeholder(tf.float32)

fc1 = tf.layers.dense(flat, units=1000)
drop1 = tf.layers.dropout(fc1, drop_prob)

fc2 = tf.layers.dense(drop1, units=500)
drop2 = tf.layers.dropout(fc2, drop_prob)

out = tf.layers.dense(fc2, units=3)

for i, var in enumerate(tf.trainable_variables()):
    #tf.summary.histogram('variable_{}'.format(i), var)
    tf.summary.histogram(var.name, var)

loss = tf.losses.softmax_cross_entropy(y_, out)
tf.summary.scalar('loss', loss)

train_op = tf.train.GradientDescentOptimizer(1e-6).minimize(loss)

pred = tf.nn.softmax(out)
accuracy = tf.metrics.accuracy(tf.argmax(y_, axis=1), tf.argmax(pred, axis=1))

merged = tf.summary.merge_all()

INFO:tensorflow:Summary name conv2d/kernel:0 is illegal; using conv2d/kernel_0 instead.
INFO:tensorflow:Summary name conv2d/bias:0 is illegal; using conv2d/bias_0 instead.
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0 is illegal; using conv2d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_2/bias:0 is illegal; using conv2d_2/bias_0 instead.
INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    coord = tf.train.Coordinator() #Coordinator는 thread를 관리하는 역할을 함
    thread = tf.train.start_queue_runners(sess, coord)
    writer = tf.summary.FileWriter('./logs/', sess.graph)
    
#    _image, _label = sess.run([x, y_])
#    for i in range(32):
#        plt.figure(i)
#        print('age: {}'.format(_label[i]))
#        plt.imshow(_image[i])

#    print(y_)

    for i in range(10000):
        _, _loss, _summaries = sess.run([train_op, loss, merged], feed_dict={drop_prob: 0.7})
        _pred, _accuracy = sess.run([pred, accuracy], feed_dict={drop_prob: 1.0})
        writer.add_summary(_summaries, i)
        
        if i%100 == 0:
            print('step: {}'.format(i))
            print('loss: {}'.format(_loss))
            print('accuracy: {}'.format(_accuracy))
            #print('prediction 1: {}'.format(tf.argmax(_pred[0])))
            #print('prediction 2: {}'.format(tf.argmax(_pred[1])))
            #print('prediction 3: {}'.format(tf.argmax(_pred[2])))
        
    coord.request_stop()
    coord.join(thread)

step: 0
loss: 6.185981750488281
accuracy: (0.0, 0.4375)
step: 100
loss: 3.403373956680298
accuracy: (0.5753125, 0.57425743)
step: 200
loss: 1.863694190979004
accuracy: (0.52109373, 0.52052242)
step: 300
loss: 2.5606775283813477
accuracy: (0.50156248, 0.50134969)
step: 400
loss: 2.368386745452881
accuracy: (0.49164063, 0.49134976)
step: 500
loss: 3.16355299949646
accuracy: (0.48725, 0.48708832)
step: 600
loss: 2.5184879302978516
accuracy: (0.4819271, 0.48180115)
step: 700
loss: 2.068319797515869
accuracy: (0.47897321, 0.47909236)
step: 800
loss: 1.756575345993042
accuracy: (0.47683594, 0.47682583)
step: 900
loss: 2.150402784347534
accuracy: (0.47427082, 0.4742994)
step: 1000
loss: 2.7075233459472656
accuracy: (0.47256249, 0.47262114)
step: 1100
loss: 2.7922558784484863
accuracy: (0.4728125, 0.47280881)
step: 1200
loss: 2.8890135288238525
accuracy: (0.47252604, 0.47241881)
step: 1300
loss: 3.0305368900299072
accuracy: (0.4725, 0.47256917)
step: 1400
loss: 1.316534399986267
accuracy: (0.4